In [ ]:
from itertools import permutations
import requests

In [47]:
import numpy as np
import pandas as pd

In [42]:
lista_artistas = [
    ['Guns N’ Roses', 'The Strokes'], # rockero
    ['Gwen Stefani', 'Lana del Rey'], # pop
    ['Travis Scott'],
    ['Martin Garrix'],
    ['Los Fabulosos Cadillacs'],
    ['DUKI'],
    ['J mena'],
    ['Louta'],
    ['James Blake'],
    ['Rels B']
]

In [112]:
roadmaps = []
c = 0
for artistas in lista_artistas:
    for features in permutations(audio_features, 4):
        for shuffle in [0,1]:
            params = {
                'artists': ','.join(artistas),
                'audio_feats': ','.join(features),
                'shuffle': shuffle     
                }
            r = requests.get('http://127.0.0.1:5000/get_roadmap', params)
            roadmaps.append(r.json()['roadmap'])

In [51]:
df = pd.read_csv('artists_lolla.csv')
df.head()

,artist,day,is_argentinian,first_release,genres,genres_clean,followers,energy,danceability,valence,instrumentalness,acousticness,speechiness,tempo,loudness,tsne_0,tsne_1,cluster_name
0,Travis Scott,1,0,2012,['rap'],['rap'],7705985,0.5976,0.7246,0.28055,2.527000e-06,0.095463,0.12278,0.557844,0.895232,-49.622486,-63.165020,Hip hop & rap
1,Martin Garrix,1,0,2011,"['big room', 'edm', 'pop', 'progressive house'...","['big room', 'edm', 'pop', 'progressive house'...",12574151,0.6256,0.5256,0.26740,1.121700e-05,0.176850,0.04220,0.561308,0.895473,23.056034,46.405690,EDM & house
2,Los Fabulosos Cadillacs,1,1,1986,"['argentine rock', 'latin alternative', 'latin...","['rock', 'alternative', 'rock', 'rock', 'ska']",1672897,0.7631,0.6917,0.81630,5.206087e-03,0.157502,0.05998,0.512656,0.910417,-27.807325,33.983185,Argentinian rock
3,Brockhampton,1,0,2015,"['boy band', 'hip hop', 'pop', 'rap']","['boy band', 'hip hop', 'pop', 'rap']",1186410,0.6064,0.7011,0.60480,5.077100e-03,0.433260,0.12062,0.525001,0.884955,-33.395466,-67.094890,Hip hop & rap
4,DUKI,1,1,2017,"['argentine hip hop', 'trap argentino']","['hip hop', 'trap']",2715207,0.6837,0.7668,0.50480,5.460000e-07,0.303400,0.16841,0.549335,0.910612,84.602455,-40.784866,Argentinian trap


In [59]:
map_dict = df[['artist', 'cluster_name']].set_index('artist').to_dict()['cluster_name']

In [113]:
cluster_counts = []
for i in range(10):
    roads = roadmaps[i*48:(i+1)*48]
    roads = np.array(roads)
    roads = pd.Series(roads.ravel())
    cluster_count = roads.map(map_dict).value_counts(normalize=True)
    cluster_counts.append(cluster_count)

In [114]:
tabla = pd.concat(cluster_counts, axis=1, sort=True)

In [115]:
tabla.index

Index(['Argentinian indie', 'Argentinian pop', 'Argentinian rock',
       'Argentinian trap', 'Art pop & R&B', 'EDM & house', 'Hip hop & rap',
       'International indie & rock', 'International pop', 'Latin trap'],
      dtype='object')

In [116]:
tabla.columns = ['International indie & rock', 'International pop', 'Hip hop & rap', 'EDM & house',
                 'Argentinian rock', 'Argentinian trap', 'Argentinian pop', 'Argentinian indie',
                 'Art pop & R&B', 'Latin trap']

In [117]:
tabla.applymap(lambda x: str(np.around(x*100,2))+'%')

,International indie & rock,International pop,Hip hop & rap,EDM & house,Argentinian rock,Argentinian trap,Argentinian pop,Argentinian indie,Art pop & R&B,Latin trap
Argentinian indie,17.14%,16.93%,16.72%,16.88%,16.41%,14.95%,16.41%,30.0%,20.57%,15.16%
Argentinian pop,3.07%,7.24%,5.1%,5.21%,6.3%,5.99%,10.0%,6.35%,6.93%,7.03%
Argentinian rock,10.94%,9.64%,6.72%,6.77%,20.0%,5.89%,8.02%,9.32%,8.75%,6.15%
Argentinian trap,1.56%,2.66%,6.93%,5.16%,6.93%,15.0%,6.67%,8.28%,5.31%,8.33%
Art pop & R&B,1.09%,2.4%,2.97%,2.97%,2.08%,3.85%,3.33%,3.8%,7.5%,2.45%
EDM & house,15.36%,11.93%,19.84%,32.5%,14.74%,17.86%,16.25%,15.52%,14.43%,15.05%
Hip hop & rap,0.21%,2.03%,10.0%,0.57%,1.98%,4.74%,2.34%,4.27%,5.31%,3.23%
International indie & rock,40.0%,13.65%,12.24%,16.82%,14.69%,9.32%,12.66%,9.53%,15.16%,9.58%
International pop,5.05%,22.5%,8.33%,4.06%,8.75%,8.54%,11.2%,7.34%,8.65%,8.02%
Latin trap,5.57%,11.04%,11.15%,9.06%,8.12%,13.85%,13.12%,5.57%,7.4%,25.0%


In [139]:
for c in tabla.columns:
    print('Perfil {}:'.format(c))
    print(tabla[c].sort_values(ascending=False)[:5].apply(lambda x: '({:.0f}%)'.format(x*100)))
    print('\n')

Perfil International indie & rock:
International indie & rock    (40%)
Argentinian indie             (17%)
EDM & house                   (15%)
Argentinian rock              (11%)
Latin trap                     (6%)
Name: International indie & rock, dtype: object


Perfil International pop:
International pop             (22%)
Argentinian indie             (17%)
International indie & rock    (14%)
EDM & house                   (12%)
Latin trap                    (11%)
Name: International pop, dtype: object


Perfil Hip hop & rap:
EDM & house                   (20%)
Argentinian indie             (17%)
International indie & rock    (12%)
Latin trap                    (11%)
Hip hop & rap                 (10%)
Name: Hip hop & rap, dtype: object


Perfil EDM & house:
EDM & house                   (32%)
Argentinian indie             (17%)
International indie & rock    (17%)
Latin trap                     (9%)
Argentinian rock               (7%)
Name: EDM & house, dtype: object


Perfil Argenti

In [125]:
tabla.to_csv('all_roadmaps.csv')

In [144]:
(np.array(roadmaps[0]) != np.array(roadmaps[1])).sum() / 40

0.25

In [146]:
diffs = []
for i in range(0,len(roadmaps),2):
    diffs.append((np.array(roadmaps[i]) != np.array(roadmaps[i+1])).sum() / 40)

In [147]:
np.mean(diffs)

0.3561458333333334